# Imports

In [7]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, Expression
from biogeme.models import loglogit
from biogeme.tools import likelihood_ratio_test
from biogeme.results import compile_estimation_results
from biogeme.models import loglogit,  boxcox
from biogeme.models.piecewise import piecewise_formula
from biogeme.models import lognested
from biogeme.nests import OneNestForNestedLogit, NestsForNestedLogit
from biogeme.biogeme import BIOGEME

import pickle

import numpy as np
import os

from scipy.stats import chi2

# Data & Variables

In [8]:
# Define the relative path to the data folder
file_path = os.path.join(os.pardir, 'lpmc01.dat')

#file_path = os.path.join(data_folder, 'lpmc01.dat')

df = pd.read_csv(file_path, sep = '\t')
df['age_normalized'] = (df['age'] - df['age'].mean()) / df['age'].std()
df['age_scaled'] = df['age'] / df['age'].max()
df['cost_driving'] = df['cost_driving_ccharge'] + df['cost_driving_fuel']
df['dur_pt'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_int'] + df['dur_pt_bus']

database1 = db.Database('lpmc01', df)


# Define the given veriables 
dur_pt = Variable('dur_pt')
cost_driving = Variable('cost_driving')
age_scaled = Variable('age_scaled')
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
trip_n = Variable('trip_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_date = Variable('travel_date')
day_of_week = Variable('day_of_week')
start_time = Variable('start_time')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable('distance')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_access = Variable('dur_pt_access') # Predicted total access and egress time for public transport route in hours
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int') # Time taken (hrs) at each interchange point
pt_interchanges = Variable('pt_interchanges')   # Number of interchange points in public transport route
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')   # Estimated fuel cost of driving route in GBP
cost_driving_ccharge = Variable('cost_driving_ccharge')  # Estimated congestion charge cost of driving route in GBP
driving_traffic_percent = Variable('driving_traffic_percent')


variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")



# Define pt_cost (not needed)
# Original paper, page 31: "Public transport fares are determined for single trips using Oystercard/contactless payment."
# Therefore, cost_transit should already consider faretype and bus_scale

database = db.Database('lpmc01', df)
variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")

# Define driving cost
cost_driving = cost_driving_ccharge + cost_driving_fuel

# Define time taken by each mode of transport
dur_pt = dur_pt_access + dur_pt_int + dur_pt_bus + dur_pt_rail  # Public transport (external) time 

'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.
'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.


# Model Definition

In [9]:
# Assume every mode of transport is available
availability_walk = 1  
availability_cycle = 1  
availability_pt = 1     
availability_drive = 1

availability = {
    1: availability_walk,   # Walking
    2: availability_cycle,  # Cycling
    3: availability_pt,     # Public Transport
    4: availability_drive   # Driving
}

In [10]:
thresholds = [0, 1, 5, None]

ALPHA_0_1 = Beta('ALPHA_0_1', 0, None, None, 0)
ALPHA_1_5 = Beta('ALPHA_1_5', 0, None, None, 0)
ALPHA_5_INF = Beta('ALPHA_5_INF', 0, None, None, 0)
betas_piecewise = [ALPHA_0_1, 
                   ALPHA_1_5, 
                   ALPHA_5_INF]

piecewise_cost_driving = piecewise_formula(
    'cost_driving', thresholds, betas_piecewise
)

piecewise_cost_pt = piecewise_formula(
    'cost_transit', thresholds, betas_piecewise
)

In [11]:
# Define alternative-specific parameters for travel time
B_TIME_WALK = Beta('B_TIME_WALK', 0, None, None, 0)
B_TIME_CYCLE = Beta('B_TIME_CYCLE', 0, None, None, 0)
B_TIME_PT = Beta('B_TIME_PT', 0, None, None, 0)
B_TIME_DRIVE = Beta('B_TIME_DRIVE', 0, None, None, 0)

ASC_CYCLE = Beta('ASC_CYCLE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

# Define generic parameters for cost and travel time
B_COST = Beta('B_COST', 0, None, None, 0)

G_TIME_WALK_AGE = Beta('G_TIME_WALK_AGE', 0, None, None, 0)
G_TIME_CYCLE_AGE = Beta('G_TIME_CYCLE_AGE', 0, None, None, 0) 
G_TIME_PT_AGE = Beta('G_TIME_PT_AGE', 0, None, None, 0)
G_TIME_DRIVE_AGE = Beta('G_TIME_DRIVE_AGE', 0, None, None, 0) 

# Utility functions with interaction terms
V_WALK = (B_TIME_WALK + G_TIME_WALK_AGE * age_scaled) * dur_walking
V_CYCLE = ASC_CYCLE + (B_TIME_CYCLE + G_TIME_CYCLE_AGE* age_scaled) * dur_cycling
V_PT = ASC_PT + piecewise_cost_pt + (B_TIME_PT + G_TIME_PT_AGE* age_scaled) * dur_pt
V_DRIVE = ASC_DRIVE +  piecewise_cost_driving + (B_TIME_DRIVE + G_TIME_DRIVE_AGE * age_scaled) * dur_driving

# Associate utility functions with the mode choice
V = {
    1: V_WALK,    # Walking
    2: V_CYCLE,   # Cycling
    3: V_PT,      # Public Transport
    4: V_DRIVE    # Driving
}


mu_a = Beta('mu_a', 1, 0, None, 0)
mu_b = Beta('mu_b', 1, 0, None, 0)

nest_a = OneNestForNestedLogit(nest_param=mu_a, list_of_alternatives=[1, 2], name='slow modes')
nest_b = OneNestForNestedLogit(nest_param=mu_b, list_of_alternatives=[3, 4], name='faster modes')
nests = NestsForNestedLogit(choice_set=list(V), tuple_of_nests=(nest_a, nest_b))

logprob_m4 = lognested(V, availability, nests, travel_mode)

model_4 = bio.BIOGEME(database, logprob_m4)
model_4.modelName = 'model_4'


# Model Results

In [12]:
results_m4 = model_4.estimate()
model_4_loglike = results_m4.data.logLike
model_4_numParam = results_m4.get_estimated_parameters().shape[0]
model_4_table = results_m4.get_estimated_parameters()

model_4_table

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ALPHA_0_1,-0.928748,0.132761,-6.995647,2.640332e-12
ALPHA_1_5,-0.019953,0.066296,-0.300960,7.634447e-01
ALPHA_5_INF,-0.267786,0.054162,-4.944179,7.646524e-07
ASC_CYCLE,-6.775249,0.320108,-21.165530,0.000000e+00
ASC_DRIVE,-2.523811,0.169161,-14.919586,0.000000e+00
ASC_PT,-2.939033,0.204898,-14.343848,0.000000e+00
B_TIME_CYCLE,-4.033973,0.992219,-4.065607,4.790759e-05
B_TIME_DRIVE,-7.593038,1.330149,-5.708413,1.140344e-08
B_TIME_PT,-3.424712,0.679065,-5.043276,4.576286e-07
B_TIME_WALK,-9.465869,0.679248,-13.935802,0.000000e+00


In [13]:
value_mu_existing = model_4_table.loc['mu_a', 'Value']
print(f'Nest parameter: {value_mu_existing:.3g}')

mu_stderr = model_4_table.loc['mu_a', 'Rob. Std err']
tested_value = 1
t_test = (tested_value - value_mu_existing) / mu_stderr
print(f't_test against 1: {t_test:.3g}')



Nest parameter: 0.556
t_test against 1: 9.87


In [14]:
print(results_m4.print_general_statistics())

Number of estimated parameters:	16
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4148.466
Likelihood ratio test for the init. model:	5566.011
Rho-square for the init. model:	0.402
Rho-square-bar for the init. model:	0.399
Akaike Information Criterion:	8328.932
Bayesian Information Criterion:	8433.208
Final gradient norm:	1.3733E-01
Nbr of threads:	16



# Test against Model 2B

In [15]:
folder_path = os.path.join(os.pardir, 'Model_3')
file_path = os.path.join(folder_path, 'model_3_piecewise.pickle')

# open a file, where you stored the pickled data
file = open(file_path, 'rb')

# dump information to that file
results_m2 = pickle.load(file)

# close the file
file.close()

In [16]:
model_4_loglike = results_m4.data.logLike
model_4_numParam = results_m4.get_estimated_parameters().shape[0]

loglikehood_m2 = results_m2.logLike
num_params_m2 = results_m2.nparam

# Calculate the LR statistic
LR = 2 * (model_4_loglike - loglikehood_m2)

# Degrees of freedom
d_f = model_4_numParam - num_params_m2

# Critical value at 0.05 significance level
critical_value = chi2.ppf(0.95, d_f)

print("Likelihood Ratio:", LR)
print("Degrees of Freedom:", d_f)
print("Critical Chi-Square Value (0.05 significance):", critical_value)

if LR > critical_value:
    print("Model 4 is significantly better than Model 3.")
else:
    print("No significant improvement in Model 4 over Model 3.")

Likelihood Ratio: 50.535501922351614
Degrees of Freedom: 2
Critical Chi-Square Value (0.05 significance): 5.991464547107979
Model 4 is significantly better than Model 3.
